## O código abaixo, até a linha anterior ao título, é apenas para facilitar o carregamento de dados no google colab e será removido futuramente

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cp "drive/My Drive/Colab Notebooks/PLN/text_sumarizer/data/news-of-the-site-folhauol.zip" "news-of-the-site-folhauol.zip"

In [0]:
cp "drive/My Drive/Colab Notebooks/PLN/text_sumarizer/models/glove_s100.zip" "glove_s100.zip"

In [0]:
cp -r "drive/My Drive/Colab Notebooks/PLN/nlputils" "nlputils"


In [0]:
cp "drive/My Drive/Colab Notebooks/PLN/text_sumarizer/requirements.txt" "requirements.txt"

In [0]:
from zipfile import ZipFile
with ZipFile("news-of-the-site-folhauol.zip", 'r') as zip_obj:
    zip_obj.extractall()
with ZipFile("glove_s100.zip", 'r') as zip_obj:
    zip_obj.extractall()

In [0]:
mv articles.csv dataset_artigos_de_noticias.csv

In [0]:
!pip3 install unidecode

In [0]:
import nltk
nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')

# Este notebook implementa o treinamento do modelo sumarizador utilzando um corpus de treinamento e word embeddings pré-treinados

## Carregando o dataset

In [0]:
import pandas as pd

In [0]:
caminho_dataset = "dataset_artigos_de_noticias.csv"
dataset_noticias = pd.read_csv(caminho_dataset)

In [15]:
# Exibe as 5 primeiras linhas do dataset carregado
dataset_noticias.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [16]:
len(dataset_noticias)

167053

In [17]:
print("Título 3:\n", dataset_noticias['title'][3], '\n')
print("Texto 3:\n", dataset_noticias['text'][3], '\n')

Título 3:
 Filme 'Star Wars: Os Últimos Jedi' ganha trailer definitivo; assista 

Texto 3:
 A Disney divulgou na noite desta segunda-feira (9) o novo trailer de "Star Wars: Os Últimos Jedi", oitavo episódio da saga.  O trailer era aguardado pelos fãs e se tornou um dos tópicos mais comentados no Twitter no horário de seu lançamento.  Assista ao trailer de 'Os Últimos Jedi'  Assista ao trailer de 'Os Últimos Jedi'  Em "O Despertar da Força" (2015), episódio mais recente, a personagem Rey (Daisy Ridley) descobre que tem a Força e procura por Luke Skywalker (Mark Hamill) para começar seu treinamento Jedi.  A história do novo episódio continua desse ponto, e cenas do trailer mostram a relação entre Rey e Skywalker.  Com direção de Rian Johnson, o filme será lançado em 14 de dezembro no Brasil. O nono episódio, ainda sem título, encerra a nova trilogia em 20 de dezembro de 2019.  O estúdio também divulgou novo poster do filme.  Poster 



## Tokenizando por palavras

Obtendo um dicionário com todas as palavras distintas e suas respectivas frequências de ocorrência no dataset

In [0]:
from nlputils.lexical import normalization as norm
normalizer = norm.Normalizer()

In [0]:
LIMITE = 10000 #len(dataset_noticias)
tokens_freqs = {} # As chaves serão as palavras e os valores serão as frequências de ocorrência
for i in range(LIMITE):
    conteudo = f"{dataset_noticias['title'][i]}\n{dataset_noticias['text'][i]}"
    tokens = normalizer.tokenize_words(conteudo)
    for token in tokens:
        if token in tokens_freqs:
            tokens_freqs[token] += 1
        else:
            tokens_freqs[token] = 1

In [21]:
len(tokens_freqs)

152021

Ordenando os tokens por frequência de ocorrência

In [0]:
from collections import OrderedDict
from operator import itemgetter    

tokens_freqs = OrderedDict(sorted(tokens_freqs.items(), key = itemgetter(1), reverse = True))

In [23]:
print([(t, tokens_freqs[t]) for t in list(tokens_freqs)[:10]])

[(',', 376121), ('de', 247513), ('.', 245652), ('a', 154082), ('que', 132219), ('o', 130963), ('e', 119377), ('do', 92425), ('da', 81786), ('em', 72624)]


## Convertendo os tokens para representações vetoriais

Carregando os vetores de palavras pré-treinados(será usado o GloVe)

In [0]:
from gensim.models import KeyedVectors
glove_path = "glove_s100.txt"
vects_glove = KeyedVectors.load_word2vec_format(glove_path)

In [0]:
# Dicionário para converter um token para vetor
token2vetor = {}

with open("glove_s100.txt", encoding='utf-8') as glove_file:
    for linha in glove_file: # Obs.: lixo será lido para a primeira linha
        valor = linha.split(' ')
        token = valor[0]
        vetor = np.asarray(valor[1:], dtype='float32')
        token2vetor[token] = vetor

In [60]:
len(token2vetor)

929606

In [61]:
token2vetor['de']

array([-0.059871, -0.26613 , -1.288952, -2.352894,  0.414237,  0.621193,
        1.073003, -0.548189, -0.643191,  0.966722,  0.869204, -0.212642,
       -1.038974, -0.356746, -0.24672 ,  0.471099,  0.049724, -0.7552  ,
        0.388665,  1.053411,  0.940066, -0.405713,  0.007156, -1.134779,
       -0.433221, -0.819711, -0.012   ,  0.032739, -0.225575,  1.153522,
        0.537173, -0.463806,  0.340472, -0.189793, -0.160288,  0.58365 ,
       -0.102506,  0.068884, -0.926536,  0.952415,  1.375001,  0.169173,
       -0.055722, -0.356511,  0.559938,  0.414697, -0.747112, -0.548186,
        0.35731 ,  0.851211,  0.036744, -0.746908, -0.305474, -0.137128,
       -1.137372, -0.514234, -0.272096, -0.622991, -0.762249, -0.625595,
        0.084059,  0.09621 ,  0.301824, -0.5573  ,  0.537335,  0.512624,
       -0.435838,  0.591526, -0.318885,  0.181027,  1.253085,  0.257935,
        0.734342,  0.245906,  2.948706,  0.0695  ,  0.484509, -0.255296,
        0.019143, -0.773821,  0.55352 , -1.709604, 

In [62]:
[token for token in list(token2vetor)[:10]]

['929605', ',', 'de', '.', 'a', 'o', 'e', 'que', 'do', 'da']

In [63]:
# Remove o lixo lido para a primeira linha
#del token2vetor['929605']

# Confirmando a exclusão
[token for token in list(token2vetor)[:10]]

[',', 'de', '.', 'a', 'o', 'e', 'que', 'do', 'da', 'em']

Calculando o número de tokens do dataset que não não estão presentes no vocabulário do modelo GloVe e que têm uma frequência de ocorrência significativa

In [38]:
qtd_tokens_faltantes = 0
freq_min = 20

for token, freq in tokens_freqs.items():
    if freq > freq_min:
        if token.lower() not in vects_glove.vocab:
            qtd_tokens_faltantes += 1

percent_tok_falt = round(qtd_tokens_faltantes/len(tokens_freqs), 2)*100
print("Tokens faltantes no modelo GloVe: ", qtd_tokens_faltantes)
print(f"Percentual de tokens faltantes: {percent_tok_falt}%")

Tokens faltantes no modelo GloVe:  529
Percentual de tokens faltantes: 0.35%


Foi usada um frequência mínima(freq_min) de ocorrência, para que tokens que não estejam presentes no modelo GloVe possam ser adicionados à word_embedding_matrix, mas eles precisam ocorrer o bastante nos textos, de forma que o modelo consiga entender seus significados

In [44]:
# Limita o vocabulário que será usado a tokens que têm uma 
# ocorrênica mínima >= freq_min ou fazem parte do vocabulário
# do modelo do glove

# Dicionário para converter tokens para inteiros
token2int = {}

# Dicionário para converter inteiros para tokens
int2token = {}

valor = 0
for token, freq in tokens_freqs.items():
    if freq >= freq_min or token.lower() in token2vetor:
        token2int[token] = valor
        int2token[valor] = token
        valor += 1

# Tokens especiais que serão adicionados ao dicionário
tokens_espec = ["<UNK>","<PAD>","<EOS>","<GO>"]

# Adiciona os tokens especiais ao vocabulário
for token in tokens_espec:
    valor = len(token2int) # Os códigos especias serão os maiores códigos
    token2int[token] = valor
    int2token[valor] = token

percent_uso_tok = round(len(token2int)/len(tokens_freqs), 4)*100    
    
print("Total de palavras distintas:", len(tokens_freqs))
print("Quantidade de palavras que serão usadas:", len(token2int))
print(f"Percentual de uso de tokens: {percent_uso_tok}%")

Total de palavras distintas: 152021
Quantidade de palavras que serão usadas: 125931
Percentual de uso de tokens: 82.84%


Construindo a matriz de tokens(embedding matrix)

In [67]:
import numpy as np

dim_vetor = 100 # Esse valor precisa conferir com o do modelo Glove
qtd_tokens = len(token2int)

# Cria a matrix com valores padrões como zero
matriz_de_tokens = np.zeros((qtd_tokens, dim_vetor), dtype=np.float32)

for token, valor in token2int.items():
    if token in token2vetor:
        matriz_de_tokens[i] = token2vetor[token]
    else:
        # Se o token não estiver no modelo GloVe, cria um vetor aleatório para ele
        novo_vetor = np.array(np.random.uniform(-1.0, 1.0, dim_vetor))
        token2vetor[token] = novo_vetor
        matriz_de_tokens[valor] = novo_vetor
        
# Confere se o valor bate com len(token2int)
print(len(matriz_de_tokens))

125931
